In [79]:
#!pip install yfinance --upgrade
#!pip install talib-binary
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd ta-lib
#!./configure --prefix=/usr
#!make
#!make install
#!pip install Ta-Lib

In [80]:
import os
import sys
import copy
import datetime
import time
import itertools

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import yfinance as yf
import talib as ta
from talib import MA_Type

**Defining Data Folder Structures**

In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
#data folder structures
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
backtest_output_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/backtest_output'

In [83]:
# define list of tickers to backtest
#ticker_list = ['ARCC', 'STAG', 'O', 'WPC', 'MAIN', 'ABR', 'ACRE', 'DIVO', 'JEPI', 'LTC', 'OHI', 'PFLT', 'QYLD', 'SCHD', 'STOR', 'XRMI', 'TQQQ', 'TSLA', 'AAPL', 'GLD', 'SLV', 'GDX']
#ticker_list = ['IRM', 'WPC', 'UVXY', 'SQQQ', 'MOS', 'ARCC', 'HTGC', 'ACRE', 'ABR', 'CTO', 'O', 'IBM', 'CAH', 'DLR', 'ORCC', 'ABBV', 'GLD', 'SLV']
#ticker_list = ['AAPL', 'GOOG', 'AMZN', 'IBM', 'MU', 'GLD', 'SLV', 'QQQ', 'IWM', 'GME']
ticker_list = ['ABR', 'ACRE', 'AGNC', 'ARCC', 'BST', 'EPD', 'EOS', 'HRZN', 'HTGC', 'KMI', 'MAIN', 'O', 'OHI', 'ORCC', 'PDI', 'PFLT', 'PNNT', 'RC', 'RITM', 'STAG', 'TCPC', 'UTG', 'VZ', 'WPC']

# define date range
start_date = '2021-03-20'
end_date = '2022-12-31'
interval = '1d'

# initial capital in USD
initial_capital = 8000

# position sizing
pos_size = round(initial_capital/len(ticker_list),2)

**2) Getting OHLC Data from Yahoo Finance** </b>


*   Get stock data based on ticker list from Yahoo Finance
*   Create each stock data as different dataframe
*   Add Technical Analysis indicators from TA-Lib
*   Pointing working directory to data folder, export dach dataframe as independent csv file 





In [84]:
# changing working directory to data folder's path
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
os.chdir(data_folder)

# getting OHLC data from yfinance package, if auto_adjust=True, OHLC data will not have adj close column, use progress=False to get rid of comments
# Other indicators: 14 day RSI
for ticker in ticker_list:
  try:
    globals()[ticker] = pd.DataFrame
    globals()[ticker] = yf.download(ticker, start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
    globals()[ticker]['50EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=50)
    globals()[ticker]['100EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=100)
    globals()[ticker]['200EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=200) 
    globals()[ticker]['ATR'] = ta.ATR(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'], timeperiod=14)
    globals()[ticker]['RSI_6'] = ta.RSI(globals()[ticker]['Close'], timeperiod=6)
    globals()[ticker]['RSI_14'] = ta.RSI(globals()[ticker]['Close'], timeperiod=14)
    globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    globals()[ticker]['ADX'] = ta.ADX(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14)
    globals()[ticker]['MFI'] = ta.MFI(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'], globals()[ticker]['Volume'], 14)
    #globals()[ticker]['3WhiteSolders'] = ta.CDL3WHITESOLDIERS(globals()[ticker]['Open'], globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'], 14)
    
    # Bollinger Bands
    globals()[ticker]['BBand_U'], globals()[ticker]['BBand_M'], globals()[ticker]['BBand_L'] = ta.BBANDS(globals()[ticker]['Close']) # if add exponential smoothing, matype=MA_Type.T3

    # Stochastic indicators (original: fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3)
    globals()[ticker]['Stoch_slowk'], globals()[ticker]['Stoch_slowd'] = ta.STOCH(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'],\
                                                                         fastk_period=21, slowk_period=5, slowk_matype=0, slowd_period=5, slowd_matype=0)    

    # add/drop columns and clean up
    #globals()[ticker] = globals()[ticker].drop(['Volume', 'MACD_hist'], axis=1)
    globals()[ticker] = globals()[ticker].round(4).dropna()
    globals()[ticker] = globals()[ticker].reset_index()
    globals()[ticker]['Position'] = 0
    globals()[ticker][['Equity Value', 'MDD_dollar', 'PNL', 'Holding Period', 'Win Count']] = np.NAN
    
    #writing each ticker dataframe into separate .csv files, save to google drive
    #globals()[ticker].to_csv(ticker+'.csv', index=False, header=True)
  except:
    print(globals()[ticker])

**Other Optional Indicators:** </b>

**MACD_hist** - globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9) </b>

**ADX** - globals()[ticker]['ADX'] = ta.ADX(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14) </b>


**ADXR** - globals()[ticker]['ADXR'] = ta.ADXR(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14) </b>

**50 Day Exponential Moving Average** - globals()[ticker]['50EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=50) </b>

**Bollinger Bands** - globals()[ticker]['BBand_U'], globals()[ticker]['BBand_M'], globals()[ticker]['BBand_L'] = ta.BBANDS(globals()[ticker]['Close']) # if add exponential smoothing, matype=MA_Type.T3


**3) Building the DataFrame List, Perform Backtesting**

In [85]:
#df_list = [ARCC, STAG, O, WPC, MAIN, ABR, ACRE, DIVO, JEPI, LTC, OHI, PFLT, QYLD, SCHD, STOR, XRMI, TQQQ, TSLA, AAPL, GLD, SLV, GDX]
#df_list = [IRM, WPC, UVXY, SQQQ, MOS, ARCC, HTGC, ACRE, ABR, CTO, O, IBM, CAH, DLR, ORCC, ABBV, GLD, SLV]
#df_list = [AAPL, GOOG, AMZN, IBM, MU, GLD, SLV, QQQ, IWM, GME]
df_list = [ABR, ACRE, AGNC, ARCC, BST, EPD, EOS, HRZN, HTGC, KMI, MAIN, O, OHI, ORCC, PDI, PFLT, PNNT, RC, RITM, STAG, TCPC, UTG, VZ, WPC]

In [86]:
# function to extract dataframe name
def get_df_name(df):
   name =[x for x in globals() if globals()[x] is df][0]
   return name

In [87]:
# function to print function name as a string
def get_func_name(func_name):
    func_name = sys._getframe().f_code.co_name
    return func_name

In [88]:
# defining the global list for storing output
global bb_macd_tradelist
bb_macd_tradelist = []

global bbadxrsi_trade_list
bbadxrsi_trade_list = []

pos_size = round(initial_capital/len(ticker_list),2)

**MACD Bollinger Bands Strategy**

In [89]:
WPC.head().T

,0,1,2,3,4
Date,2022-01-03 00:00:00,2022-01-04 00:00:00,2022-01-05 00:00:00,2022-01-06 00:00:00,2022-01-07 00:00:00
Open,79.0445,78.17,78.4583,77.603,77.2858
High,79.0445,78.7658,78.8811,77.8432,77.7375
Low,76.8053,78.1315,77.0744,77.1224,77.1032
Close,78.0931,78.5159,77.3627,77.478,77.5165
Volume,1364400,802900,853100,712500,572900
50EMA,75.1838,75.3145,75.3948,75.4765,75.5565
100EMA,73.7779,73.8717,73.9408,74.0109,74.0803
200EMA,71.7003,71.7682,71.8238,71.8801,71.9362
ATR,1.2224,1.1831,1.2277,1.1914,1.1517


In [90]:
def backtest_bbmacd(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  profit_target = initial_equity_value * 1.06

  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_100EMA      = df.loc[i, '100EMA']
    now_200EMA      = df.loc[i, '200EMA']
    now_rsi_6       = df.loc[i, 'RSI_6']
    now_rsi_14      = df.loc[i, 'RSI_14']
    now_MFI         = df.loc[i, 'MFI']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False) and ((now_open > now_bband_m) and ((now_MACD > now_MACD_signal))):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.loc[i, 'Position'] = 1
      df.loc[i, 'Equity Value'] = initial_equity_value
      bb_macd_tradelist.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])
    
    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif ((pos_opened == True) and ((now_open > now_bband_u) or (now_MACD < now_MACD_signal))) or \
      ((pos_opened == True) and (now_date == df.loc[(len(df)-1), 'Date'])):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      pnl = round(((close_price - initial_buy_price) * lot_size) - sell_commission,2)
      df.loc[i, 'Position'] = 2
      df.loc[i, 'Equity Value'] = (now_open * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - initial_equity_value
      df.loc[i, 'PNL'] = pnl.round(2)
      df.loc[i, 'Holding period'] = hold_counter
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      bb_macd_tradelist.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      hold_counter = 0

    # calculating daily drawdowns
    #else (pos_opened == True):
    elif (pos_opened==True):
      df.loc[i, 'Equity Value']   = (now_close * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - round(initial_equity_value,2)
      hold_counter = hold_counter + 1
 
  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].max().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

In [91]:
bbmacd_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_bbmacd(df)
    bbmacd_result_list.append(test_result)

    bbmacd_result_df = pd.DataFrame(bbmacd_result_list)
    bbmacd_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = bbmacd_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    bbmacd_result_df = pd.DataFrame(bbmacd_result_list)
    bbmacd_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [92]:
# PNL for Bollinger Bands + MACD Strategy

#measure_date = df_list[1].loc[1, 'Date']
#end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: Bollinger Bands + MACD')
#print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((bbmacd_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = bbmacd_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (bbmacd_result_df['No of Trade'].sum()))
print('Total PNL: USD', bbmacd_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(bbmacd_result_df[bbmacd_result_df['No of Trade'] > 0]['Win Rate %'].mean(),2), '%')
bbmacd_result_df[bbmacd_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)


Trading Strategy: Bollinger Bands + MACD
Total No of Trades Executed: 276
Total PNL: USD 1853.31
Required Capital: USD 8000
ROI:  23.17 %
Avg Win Rate:  58.32 %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
9,KMI,10,130.56,13.06,50.64,13.60,70.00
17,RC,8,127.43,15.93,84.63,15.12,75.00
20,TCPC,15,119.09,7.94,57.71,7.67,66.67
2,AGNC,10,114.67,11.47,84.05,12.90,70.00
7,HRZN,11,112.30,10.21,83.67,11.36,63.64
0,ABR,14,107.12,7.65,69.74,7.57,50.00
10,MAIN,11,104.19,9.47,82.33,9.45,45.45
1,ACRE,11,101.38,9.22,51.00,11.18,36.36
8,HTGC,12,100.08,8.34,50.98,8.92,66.67
11,O,6,99.86,16.64,46.83,20.33,50.00


In [93]:
def backtest_bbadxrsi(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  profit_target = initial_equity_value * 1.06

  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_adx         = df.loc[i, 'ADX']
    now_rsi         = df.loc[i, 'RSI']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False) and ((now_open < now_bband_l) and (now_adx > 20) and (now_rsi < 35)):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.loc[i, 'Position'] = 1
      df.loc[i, 'Equity Value'] = initial_equity_value
      bbadxrsi_trade_list.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])
    
    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif ((pos_opened == True) and ((now_open > now_bband_u) or now_rsi > 65)) or \
      ((pos_opened == True) and (now_date == df.loc[(len(df)-1), 'Date'])):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      pnl = round(((close_price - initial_buy_price) * lot_size) - sell_commission,2)
      df.loc[i, 'Position'] = 2
      df.loc[i, 'Equity Value'] = (now_open * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - initial_equity_value
      df.loc[i, 'PNL'] = pnl.round(2)
      df.loc[i, 'Holding period'] = hold_counter
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      bbadxrsi_trade_list.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      hold_counter = 0

    # calculating daily drawdowns
    #else (pos_opened == True):
    elif (pos_opened==True):
      df.loc[i, 'Equity Value']   = (now_close * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - round(initial_equity_value,2)
      hold_counter = hold_counter + 1
 
  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].max().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

In [94]:
bbadxrsi_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_bbadxrsi(df)
    bbadxrsi_result_list.append(test_result)

    bbadxrsi_result_df = pd.DataFrame(bbadxrsi_result_list)
    bbadxrsi_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = bbadxrsi_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    bbadxrsi_result_df = pd.DataFrame(bbadxrsi_result_list)
    bbadxrsi_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  

In [95]:
# PNL for Bollinger Bands + ADX + RSI Strategy

measure_date = df_list[1].loc[1, 'Date']
end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: Bollinger Bands + ADX + RSI')
print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((bbadxrsi_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = bbadxrsi_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (bbadxrsi_result_df['No of Trade'].sum()))
print('Total PNL: USD', bbadxrsi_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(bbadxrsi_result_df[bbadxrsi_result_df['No of Trade'] > 0]['Win Rate %'].mean(),2), '%')
bbadxrsi_result_df[bbadxrsi_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)

Trading Strategy: Bollinger Bands + ADX + RSI
Trade period: 2022-01-04 to 2022-12-09
Total No of Trades Executed: 0
Total PNL: USD 0
Required Capital: USD 8000
ROI:  0.0 %
Avg Win Rate:  nan %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
